In [2]:
from Bio import SeqIO
from Bio import SeqUtils
from Bio.Seq import Seq
import os
import numpy as np
import pandas as pd
pd.set_option("display.float_format",lambda x:'%.2f'%x) # 取消科学计数显示
import time
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from matplotlib import pyplot as plt
import seaborn as sns

2024-09-25 13:14:34


In [3]:
edta_dic={"LTR/Copia":"Copia","LTR/Gypsy":"Gypsy","LTR/LINE":"LINE","DNA/DTA":"hAT", "DNA/DTC":"CACTA", 'LTR/unknown':'LTR Unknown',
          "Unknown":"Unknown",'pararetrovirus':'Other','LINE/unknown':"LINE",'polinton':'Other', 'knob/knob180':'Other','LINE/L1':"LINE",
          'subtelomere/4-12-1':'Other',"MITE/DTA":"hAT", "MITE/DTC":"CACTA", "MITE/DTH":"PIF-Harbinger", "MITE/DTM":"Mutator",
          "MITE/DTT":"Tc1_Mariner","MITE/Helitron":"Helitron",
          "DNA/DTH":"PIF-Harbinger", "DNA/DTM":"Mutator", "DNA/DTT":"Tc1_Mariner","DNA/Helitron":"Helitron",'TIR/unknown':"TIR Unknown"}
TPSI_dic={'TY1_Copia':"Copia", 'gypsy':"Gypsy", 'LINE':"LINE",'ltr_Roo':"LTR_Roo", 'hAT':"hAT",'cacta':"CACTA", 'MuDR_A_B':"Mutator",
          'mariner':"Mariner",'helitronORF':"Helitron"}

In [4]:
def get_edta_id(x):
    values=x.split(";")
    name=''
    c=''
    for value in values:
        if value.startswith("Name="):
            name=value.split('=')[1].strip()
        if value.startswith("Classification="):
            c=value.split('=')[1].strip()
    return name+"#"+c
def get_identity(x):
    if 'homology' in x:
        v=x.split("Identity=")[1].split(";")[0]
        if v=='NA':
            return 0.5
        else:
            return float(v)
    elif 'structural' in x:
        #print(x)
        if 'Identity=' in x:
            v=x.split("Identity=")[1].split(";")[0]
            if v=='NA':
                return 1
            else:
                return float(v)
        elif "identity=" in x:
            v=x.split("identity=")[1].split(";")[0]
            if v=='NA':
                return 1
            else:
                return float(v)
        else:
            v=1
        return v
    else:
        print(x)
        return 0.5


### Read the sample_list

In [6]:
samples=pd.read_excel("D:\\18.TE_Evolution\\00.Sample_Information\\Plant_558_Genome_Infor_this_project_2024-08-16.xlsx").fillna("")
samples=samples.rename(columns={"organism_name":"Name"})
print(samples.shape)
samples.head()

(558, 6)


,Name,Specie_ID,Taxonomy_ID,Assembly_Accession,Assembly,Genome_Source
0,Acer negundo,PBXX,4023,GCA_025594385.1,ASM2559438v1,NCBI
1,Acer saccharum,QIBM,4024,GCA_030979865.1,UCONN_Acsa_2,NCBI
2,Acorus calamus,BUKK,4465,GCA_030737845.1,ASM3073784v1,NCBI
3,Acorus gramineus,GJNO,55184,GCA_030737835.1,ASM3073783v1,NCBI
4,Actinidia chinensis,WWFJ,1590841,GCA_003024255.1,Red5_PS1_1.69.0,Ensemble


### Clean the results of DupGene_Finder

In [ ]:
save_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\Clean\\'
count=0
for i in range(samples.shape[0]): # New Plant 13
    specie_id=samples.loc[i,'Specie_ID']
    specie_name=samples.loc[i,'Name'].replace(" ","_")
    specie=samples.loc[i,'Name']#.replace(" ","_")
    r=[]
    file_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\DupGene\\Plant_New50\\'
    for query in os.listdir(file_path):
        if query.startswith(specie_id):
            files=os.listdir(file_path+query)
            for f in files:
                if f.endswith(".pairs"):
                    df=pd.read_csv(file_path+query+"//"+f,sep='\t')
                    pair_type=f.split(".")[-2]
                    cols=list(df.columns)
                    df['Dup_Type']=pair_type
                    df['Gene_1']=df[cols[0]]#.apply(lambda x:x.split("-RA")[0])
                    df['Gene_2']=df[cols[2]]#.apply(lambda x:x.split("-RA")[0])
                    df=df[['Gene_1','Gene_2','Dup_Type']]
                    r.append(df)
    r=pd.concat(r)
    r['Pair']=r['Gene_1'].apply(lambda x:str(x))+":"+r['Gene_2'].apply(lambda x:str(x))
    pair_dic={}
    for p,p_df in r.groupby("Pair"):
        #print(p)
        dup_type=Counter(p_df['Dup_Type'].tolist()).most_common(1)[0][0]
        pair_dic[p]=dup_type
    r=pd.DataFrame.from_dict(pair_dic,orient='index').reset_index()
    r.columns=['Gene_Pair','Dup_Type']
    count+=1
    print(count,specie_id,r['Dup_Type'].value_counts())
    r.to_csv(save_path+specie_id+"_dup.csv",index=False)

### Stat Results of DupGene_Finder

In [8]:
save_path='D://18.TE_Evolution//07.DupGene_Finder//Clean//'
count=0
R=[]
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    specie_name=samples.loc[i,'Name']#.replace(" ","_")
    specie=samples.loc[i,'Name']#.replace(" ","_")
    #sample=samples.loc[i,'Assembly'] #ZZAX_protein.fa
    dup=pd.read_csv(save_path+specie_id+"_dup.csv")
    df0=pd.DataFrame(dup['Dup_Type'].value_counts()).T.reset_index()
    df0['Specie_ID']=specie_id
    df0['Name']=specie_name
    R.append(df0)
R=pd.concat(R)
R.head()

0
100
200
300
400
500


Dup_Type,index,dispersed,transposed,proximal,tandem,wgd,Specie_ID,Name
0,count,23454,6506.00,2666,2339,1998.00,PBXX,Acer negundo
0,count,27443,8107.00,3729,3058,2087.00,QIBM,Acer saccharum
0,count,30068,5592.00,3048,2229,25317.00,BUKK,Acorus calamus
0,count,17079,5823.00,1487,1598,3345.00,GJNO,Acorus gramineus
0,count,22660,4925.00,368,1528,19764.00,WWFJ,Actinidia chinensis


In [9]:
R=R[['Specie_ID','Name','transposed', 'proximal', 'tandem', 'wgd','dispersed']]
R['Total']=R[['transposed', 'proximal', 'tandem', 'wgd','dispersed']].sum(axis=1)
for col in ['transposed', 'proximal', 'tandem', 'wgd','dispersed']:
    R[col+"(%)"]=R[col]/R["Total"]*100
    print(col,int(R[col].sum()), round(int(R[col].sum())/int(R['Total'].sum())*100,2))
R.head()

transposed 3688493 10.94
proximal 1015256 3.01
tandem 4883972 14.48
wgd 5457283 16.18
dispersed 18677688 55.39


Dup_Type,Specie_ID,Name,transposed,proximal,tandem,wgd,dispersed,Total,transposed(%),proximal(%),tandem(%),wgd(%),dispersed(%)
0,PBXX,Acer negundo,6506.00,2666,2339,1998.00,23454,36963.00,17.60,7.21,6.33,5.41,63.45
0,QIBM,Acer saccharum,8107.00,3729,3058,2087.00,27443,44424.00,18.25,8.39,6.88,4.70,61.78
0,BUKK,Acorus calamus,5592.00,3048,2229,25317.00,30068,66254.00,8.44,4.60,3.36,38.21,45.38
0,GJNO,Acorus gramineus,5823.00,1487,1598,3345.00,17079,29332.00,19.85,5.07,5.45,11.40,58.23
0,WWFJ,Actinidia chinensis,4925.00,368,1528,19764.00,22660,49245.00,10.00,0.75,3.10,40.13,46.01


In [10]:
print(int(R["Total"].sum()))

33722692


In [11]:
R.to_excel("D://18.TE_Evolution//07.DupGene_Finder//Plant_558_DupgeneFinder_GeneDup_Stats.xlsx",index=False)

### Filter Results of DupGene Finder Using TE information

In [12]:
#!pip install python-louvain
file_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\Clean\\'
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import community  # 需要安装 python-louvain 库 !pip install python-louvain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
def get_region_te_seqs(region):
    c1=region.split(":")[0]
    s1=int(region.split(":")[1].split("-")[0])
    e1=int(region.split(":")[1].split("-")[1])
    if c1 in TE_dic:
        te=TE_dic[c1]
        df=te[(te["Start"]>=s1)&(te['End']<=e1)]
        if df.shape[0]>0:
            seq1=df['TE_Type'].tolist()
            return " ".join(seq1)
    return ''
def get_region_tepep_seqs(region):
    c1=region.split(":")[0]
    s1=int(region.split(":")[1].split("-")[0])
    e1=int(region.split(":")[1].split("-")[1])
    if c1 in TEpep_dic:
        tepep=TEpep_dic[c1]
        df=tepep[(tepep["Start"]>=s1)&(tepep['End']<=e1)]
        if df.shape[0]>0:
            seq1=df['TE_Type'].tolist()
            return " ".join(seq1)
    return ''

In [14]:
def get_Gene_Pair_COS_Sim(pair):
    seq1=pair.split(":")[0]
    seq2=pair.split(":")[1]
    if (seq1 !=' ') &(seq2!=' '):
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([seq1, seq2])
        # 计算余弦相似度
        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
        return str(cosine_sim[0][0])+":"+seq1+"; "+seq2
    else:
        return 0

In [15]:
stat=pd.read_excel("D://18.TE_Evolution//07.DupGene_Finder//Plant_558_DupgeneFinder_GeneDup_Stats.xlsx")
stat.head()

,Specie_ID,Name,transposed,proximal,tandem,wgd,dispersed,Total,transposed(%),proximal(%),tandem(%),wgd(%),dispersed(%)
0,PBXX,Acer negundo,6506.00,2666,2339,1998.00,23454,36963,17.60,7.21,6.33,5.41,63.45
1,QIBM,Acer saccharum,8107.00,3729,3058,2087.00,27443,44424,18.25,8.39,6.88,4.70,61.78
2,BUKK,Acorus calamus,5592.00,3048,2229,25317.00,30068,66254,8.44,4.60,3.36,38.21,45.38
3,GJNO,Acorus gramineus,5823.00,1487,1598,3345.00,17079,29332,19.85,5.07,5.45,11.40,58.23
4,WWFJ,Actinidia chinensis,4925.00,368,1528,19764.00,22660,49245,10.00,0.75,3.10,40.13,46.01


In [ ]:
window=2000
for i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+"_dup.csv" in os.listdir(file_path):
        genedup=pd.read_csv(file_path+specie_id+"_dup.csv")
        genedup=genedup[genedup['Dup_Type'].apply(lambda x:1 if x in ['dispersed','transposed'] else 0)==1].reset_index(drop=True)
        if i==68:
            genedup['Source']=genedup['Gene_Pair'].apply(lambda x:":".join(x.split(":")[:2]))
            genedup['Target']=genedup['Gene_Pair'].apply(lambda x:":".join(x.split(":")[-2:]))
        else:
            genedup['Source']=genedup['Gene_Pair'].apply(lambda x:x.split(":")[0])
            genedup['Target']=genedup['Gene_Pair'].apply(lambda x:x.split(":")[1])
        dupgenes={}
        for gene in genedup['Source'].unique():
            dupgenes[gene]=1
        for gene in genedup['Target'].unique():
            dupgenes[gene]=1
        bed_path='D:\\18.TE_Evolution\\06.Bed\\'
        TE=pd.read_csv(bed_path+"TE_Bed\\"+specie_id+"_EDTA.bed",sep='\t')
        TE_dic={}
        for chrom,c_df in TE.groupby("#CHROM"):
            TE_dic[str(chrom)]=c_df
        TEpep=pd.read_csv(bed_path+"TEpep_Bed\\"+specie_id+"_TPSI.bed",sep='\t')
        TEpep_dic={}
        for chrom,c_df in TEpep.groupby("#CHROM"):
            TEpep_dic[str(chrom)]=c_df
        del TE
        del TEpep
        gene_bed=pd.read_csv(bed_path+"Gene_Bed_with_Strand\\"+specie_id+"_Gene.bed",sep='\t')
        gene_bed['Gene']=gene_bed['Gene'].apply(lambda x:str(x))
        gene_bed=gene_bed[gene_bed['Gene'].apply(lambda x:1 if x in dupgenes else 0)==1].reset_index(drop=True)
        #gene_bed['Start']=gene_bed['Start'].apply(lambda x: x-10000 if x>10000 else x)
        gene_bed['Start']=gene_bed['Start'].apply(lambda x: x-window if x>window else x)
        if i==195:
            #gene_bed['End']=gene_bed['End'].apply(lambda x:int(str(x).split(" ")[0])+10000)
            gene_bed['End']=gene_bed['End'].apply(lambda x:int(str(x).split(" ")[0])+window)
        else:
            gene_bed['End']=gene_bed['End']+window
        gene_bed['Region']=gene_bed['#CHROM'].apply(lambda x:str(x))+":"+gene_bed['Start'].apply(lambda x:str(x))+'-'+gene_bed['End'].apply(lambda x:str(x))
        gene_bed['TE_Seq']=gene_bed['Region'].apply(get_region_te_seqs)
        gene_bed['TEpep_Seq']=gene_bed['Region'].apply(get_region_tepep_seqs)
        print("Raw:\t",specie_id,genedup.shape)
        dupgenes={}
        for j in range(gene_bed.shape[0]):
            dupgenes[gene_bed.loc[j,'Gene']]={}
            dupgenes[gene_bed.loc[j,'Gene']]['TE_Seq']=gene_bed.loc[j,'TE_Seq']
            dupgenes[gene_bed.loc[j,'Gene']]['TEpep_Seq']=gene_bed.loc[j,'TEpep_Seq']
        genedup['Seq1']=genedup['Source'].apply(lambda x:str(dupgenes[x]['TE_Seq'])+" "+str(dupgenes[x]['TEpep_Seq']) if x in dupgenes else '')
        genedup['Seq2']=genedup['Target'].apply(lambda x:str(dupgenes[x]['TE_Seq'])+" "+str(dupgenes[x]['TEpep_Seq']) if x in dupgenes else '')
        genedup=genedup[(genedup["Seq1"]!=' ')&(genedup["Seq2"]!=" ")]
        genedup['Seq']=genedup['Seq1']+":"+genedup['Seq2']
        genedup['Cos_Sim']=genedup['Seq'].apply(get_Gene_Pair_COS_Sim)
        genedup['Cos_Sim2']=genedup['Cos_Sim'].fillna("0:").apply(lambda x:float(x.split(":")[0]))
        genedup=genedup[genedup['Cos_Sim2']>0.5]
        print(specie_id,'Round 2 \t',genedup.shape[0])#r0.shape[0],
        genedup.to_csv('D:\\18.TE_Evolution\\07.DupGene_Finder\\Filtered(2K)\\'+specie_id+"_Dups.csv",index=False)
        df = genedup[['Source','Target']]
        # 创建有向图
        G = nx.from_pandas_edgelist(df, 'Source', 'Target', create_using=nx.DiGraph())
            # 找到所有的连接模块
            # 使用社区检测算法（例如，Louvain 算法）
        partition = community.best_partition(G.to_undirected())
            # 将模块信息存储在 DataFrame 中
        modules = pd.DataFrame(list(partition.items()), columns=['Node', 'Module'])
        modules.to_csv('D:\\18.TE_Evolution\\07.DupGene_Finder\\Filtered(2K)\\'+specie_id+"_Dups_Modules.csv",index=False)

### Stat Dup Gene List

In [ ]:
file_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\Filtered(2K)\\'
save_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
total_count=0
for i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    dup_genes=[]
    if specie_id+"_Dups_Modules.csv" in os.listdir(file_path):
            df=pd.read_csv(file_path+specie_id+"_Dups_Modules.csv")
            dup_genes+=df['Node'].tolist()
            #df=df[df[3]==df[10]] # 方向相同
    if dup_genes!=[]:
        with open(save_path+specie_id+".TE_DupGenes.txt",'w') as f:
            for gene in list(set(dup_genes)):
                f.write(str(gene)+'\n')
                total_count+=1
            f.close()
total_count

In [ ]:
def get_overlap(x):
    seq1=x.split(":")[0].split(" ")
    seq2=x.split(":")[0].split(" ")
    overlap=list(set(seq1).intersection(set(seq2)))
    if '' in overlap:
        overlap.remove("")
    return " | ".join(overlap)

In [ ]:
window='2K'
file_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\Filtered('+window+')\\'
save_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes('+window+')\\Super_Family\\'
tepep_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Super_Family\\'
families=['Copia','Gypsy', 'CACTA', 'hAT','Tc1_Mariner','Mutator', 'Helitron', 'PIF-Harbinger']
Dup_genes_stat={}
te_gene_dic={}
tepep_gene_dic={}
for i in range(samples.shape[0]):
    if i%10==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    for f in os.listdir(tepep_path):
        if specie_id in f:
            family=f.split(specie_id)[0][:-1]
            for gene in [gene.strip() for gene in open(tepep_path+f,'r').readlines()]:
                tepep_gene_dic[gene]=family
    
    if specie_id+".TE_vs_Gene.bed" in os.listdir("D:\\18.TE_Evolution\\04.TE\Gene_TE\\"):
        te=pd.read_csv("D:\\18.TE_Evolution\\04.TE\Gene_TE\\"+specie_id+".TE_vs_Gene.bed",sep='\t',header=None)
        for g,g_df in te.groupby(13):
            cs=Counter(g_df[7].tolist())
            te_gene_dic[g]=cs.most_common(1)[0][0]
            break
    df=pd.read_csv(file_path+specie_id+"_Dups.csv")
    df['Seq']=df["Seq1"]+":"+df["Seq2"]
    df['Overlap']=df["Seq"].apply(get_overlap)
    df0=df[df['Overlap']!=''].reset_index(drop=True)
    df0=df0[['Source','Target','Overlap']]
    df0['Source_TE_Class']=df0['Source'].apply(lambda x:te_gene_dic[x] if x in te_gene_dic else '')
    df0['Target_TE_Class']=df0['Target'].apply(lambda x:te_gene_dic[x] if x in te_gene_dic else '')
    df0['Source_TEpep_Class']=df0['Source'].apply(lambda x:tepep_gene_dic[x] if x in tepep_gene_dic else '')
    df0['Target_TEpep_Class']=df0['Target'].apply(lambda x:tepep_gene_dic[x] if x in tepep_gene_dic else '')
    for i in range(df0.shape[0]):
        if df0.loc[i,'Source_TE_Class']!='':
            if df0.loc[i,'Source_TE_Class']==df0.loc[i,'Target_TE_Class']:
                df0.loc[i,'Overlap']=df0.loc[i,'Source_TE_Class']
        elif df0.loc[i,'Source_TEpep_Class']!='':
            if df0.loc[i,'Source_TEpep_Class']==df0.loc[i,'Target_TEpep_Class']:
                df0.loc[i,'Overlap']=df0.loc[i,'Source_TEpep_Class']
    df1=df0[(df0['Overlap']==df0['Source_TE_Class'])|(df0['Overlap']==df0['Source_TEpep_Class'])]
    df2=df0[(df0['Overlap']!=df0['Source_TE_Class'])&(df0['Overlap']!=df0['Source_TEpep_Class'])]
    df0=pd.concat([df1,df2])
            
    for family in families:
        if family not in Dup_genes_stat:
            Dup_genes_stat[family]=0
        df1=df0[df0['Overlap'].apply(lambda x:1 if family in x else 0)==1]
        family_dup_genes=list(set(df1['Source'].tolist()+df1['Target'].tolist()))
        Dup_genes_stat[family]+=len(family_dup_genes)
        with open(save_path+family+"_"+specie_id+".TE_DupGenes.txt",'w') as f:
            for gene in family_dup_genes:
                f.write(str(gene)+'\n')
            f.close()

In [ ]:
df0=pd.DataFrame.from_dict(Dup_genes_stat,orient='index').reset_index()
df0.columns=['TE_Super_Family','DupGene_Number']
df0=df0.sort_values("DupGene_Number",ascending=False)
df0.to_excel("D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes("+window+")\\Super_Family_Stats("+window+").xlsx",index=False)